In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [9]:
url = 'https://fbref.com/en/comps/9/2023-2024/schedule/2023-2024-Premier-League-Scores-and-Fixtures'

In [11]:
df = pd.read_html('https://fbref.com/en/comps/9/2023-2024/schedule/2023-2024-Premier-League-Scores-and-Fixtures', 
                 attrs={"id":"sched_2023-2024_9_1"}
                 )

In [12]:
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")


## First exploration

In [25]:
table = soup.find(id='sched_2023-2024_9_1').find("tbody")

In [32]:
table.find("tr")

<tr><th class="right" data-stat="gameweek" scope="row">1</th><td class="left" csk="6" data-stat="dayofweek">Fri</td><td class="left" csk="20230811" data-stat="date"><a href="/en/matches/2023-08-11">2023-08-11</a></td><td class="right" csk="20:00:00" data-stat="start_time"><span class="venuetime" data-venue-epoch="1691780400" data-venue-time="20:00" data-venue-time-only="1">20:00</span> <span class="localtime" data-label="your time"></span></td><td class="right" data-stat="home_team"><a href="/en/squads/943e8050/2023-2024/Burnley-Stats">Burnley</a></td><td class="right" data-stat="home_xg">0.3</td><td class="center" data-stat="score"><a href="/en/matches/3a6836b4/Burnley-Manchester-City-August-11-2023-Premier-League">0–3</a></td><td class="right" data-stat="away_xg">1.9</td><td class="left" data-stat="away_team" style="font-weight: bold;"><a href="/en/squads/b8fd03ef/2023-2024/Manchester-City-Stats">Manchester City</a></td><td class="right" csk="21572" data-stat="attendance">21,572</td>

In [46]:
table.find("tr").find('td', class_='right', attrs={'data-stat': 'home_team'}).string

'Burnley'

In [52]:
table.find("tr").find('td', class_='left', attrs={'data-stat': 'match_report'}).find('a').get('href')

'/en/matches/3a6836b4/Burnley-Manchester-City-August-11-2023-Premier-League'

In [54]:
for game in table.find_all("tr"):
    ht = game.find('td', class_='right', attrs={'data-stat': 'home_team'}).string
    print(ht)

Burnley
Arsenal
Everton
Sheffield Utd
Brighton
Bournemouth
Newcastle Utd
Brentford
Chelsea
Manchester Utd
None
Nott'ham Forest
Fulham
Liverpool
Wolves
Tottenham
Manchester City
Aston Villa
West Ham
Crystal Palace
None
Chelsea
Bournemouth
Everton
Brentford
Manchester Utd
Arsenal
Brighton
Sheffield Utd
Burnley
Newcastle Utd
None
Luton Town
Sheffield Utd
Chelsea
Manchester City
Brentford
Burnley
Brighton
Liverpool
Crystal Palace
Arsenal
None
Wolves
Fulham
West Ham
Aston Villa
Manchester Utd
Tottenham
Newcastle Utd
Bournemouth
Everton
Nott'ham Forest
None
Manchester City
Luton Town
Crystal Palace
Brentford
Burnley
Arsenal
Brighton
Chelsea
Liverpool
Sheffield Utd
None
Aston Villa
Wolves
Bournemouth
Everton
Newcastle Utd
West Ham
Manchester Utd
Tottenham
Nott'ham Forest
Fulham
None
Luton Town
None
Luton Town
Everton
Manchester Utd
Burnley
Fulham
Crystal Palace
West Ham
Wolves
Brighton
Arsenal
None
Liverpool
Bournemouth
Newcastle Utd
Brentford
Manchester City
Nott'ham Forest
Chelsea
Sheffield

In [59]:
i=0
for game in table.find_all("tr"):
    if "spacer" in game.get('class', []) and "partial_table" in game.get('class', []) and "result_all" in game.get('class', []):
        continue
    mr = game.find('td', class_='left', attrs={'data-stat': 'match_report'}).find('a').get('href')
    i+=1 
    print(i, mr)

1 /en/matches/3a6836b4/Burnley-Manchester-City-August-11-2023-Premier-League
2 /en/matches/26a7f90c/Arsenal-Nottingham-Forest-August-12-2023-Premier-League
3 /en/matches/15addfc7/Everton-Fulham-August-12-2023-Premier-League
4 /en/matches/55fd92c7/Sheffield-United-Crystal-Palace-August-12-2023-Premier-League
5 /en/matches/56a137f7/Brighton-and-Hove-Albion-Luton-Town-August-12-2023-Premier-League
6 /en/matches/d6bbf293/Bournemouth-West-Ham-United-August-12-2023-Premier-League
7 /en/matches/8ff2f8fe/Newcastle-United-Aston-Villa-August-12-2023-Premier-League
8 /en/matches/67ed3ba2/Brentford-Tottenham-Hotspur-August-13-2023-Premier-League
9 /en/matches/c18d3207/Chelsea-Liverpool-August-13-2023-Premier-League
10 /en/matches/f1ecda2c/Manchester-United-Wolverhampton-Wanderers-August-14-2023-Premier-League
11 /en/matches/a0a93f71/Nottingham-Forest-Sheffield-United-August-18-2023-Premier-League
12 /en/matches/46be83af/Fulham-Brentford-August-19-2023-Premier-League
13 /en/matches/80bebdbb/Liverpo

## Get proper table

In [106]:
fixtures = df[0].dropna(how='all')
fixtures.head(5)

KeyError: 0

In [107]:
fixtures.info()

<class 'pandas.core.frame.DataFrame'>
Index: 380 entries, 0 to 422
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   game         380 non-null    int64         
 1   date         380 non-null    datetime64[ns]
 2   home_team    380 non-null    object        
 3   score        380 non-null    object        
 4   away_team    380 non-null    object        
 5   report_link  342 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 20.8+ KB


In [108]:
fixtures['Wk'] = fixtures['Wk'].astype(int)
fixtures['Date'] = pd.to_datetime(fixtures['Date'])
fixtures = fixtures[['Wk', 'Date', 'Home', 'Score', 'Away']]
fixtures.rename(columns={'Wk': 'game',
                         'Date': 'date',
                         'Home': 'home_team',
                         'Score': 'score',
                         'Away': 'away_team'}, inplace=True)

fixtures['home_team'] = fixtures['home_team'].astype(str)
fixtures['away_team'] = fixtures['away_team'].astype(str)
                         
fixtures.head(10)

KeyError: 'Wk'

In [109]:
table = soup.find(id='sched_2023-2024_9_1').find("tbody")
match_reports = []

# Loop through the rows of the table
for game in table.find_all("tr"):
    # Skip rows with spacer
    if "spacer" in game.get('class', []) and "partial_table" in game.get('class', []) and "result_all" in game.get('class', []):
        continue

    # Extract match report URL
    match_report = game.find('td', class_='left', attrs={'data-stat': 'match_report'})
    
    # Check if the match_report element is found
    if match_report:
        mr = match_report.find('a').get('href')
        match_reports.append('https://fbref.com'+mr)

# Create a DataFrame from the list of match report URLs
df_rl = pd.DataFrame(match_reports, columns=['report_link'])



In [110]:
fixtures['report_link'] = df_rl
fixtures.head(10)

,game,date,home_team,score,away_team,report_link
0,1,2023-08-11,Burnley,0–3,Manchester City,https://fbref.com/en/matches/3a6836b4/Burnley-...
1,1,2023-08-12,Arsenal,2–1,Nott'ham Forest,https://fbref.com/en/matches/26a7f90c/Arsenal-...
2,1,2023-08-12,Everton,0–1,Fulham,https://fbref.com/en/matches/15addfc7/Everton-...
3,1,2023-08-12,Sheffield Utd,0–1,Crystal Palace,https://fbref.com/en/matches/55fd92c7/Sheffiel...
4,1,2023-08-12,Brighton,4–1,Luton Town,https://fbref.com/en/matches/56a137f7/Brighton...
5,1,2023-08-12,Bournemouth,1–1,West Ham,https://fbref.com/en/matches/d6bbf293/Bournemo...
6,1,2023-08-12,Newcastle Utd,5–1,Aston Villa,https://fbref.com/en/matches/8ff2f8fe/Newcastl...
7,1,2023-08-13,Brentford,2–2,Tottenham,https://fbref.com/en/matches/67ed3ba2/Brentfor...
8,1,2023-08-13,Chelsea,1–1,Liverpool,https://fbref.com/en/matches/c18d3207/Chelsea-...
9,1,2023-08-14,Manchester Utd,1–0,Wolves,https://fbref.com/en/matches/f1ecda2c/Manchest...


In [111]:
table.find("tr", data_row="0")

In [62]:
df[0].tail(15)

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
409,34.0,Tue,2024-05-14,20:00,Tottenham,1.7,0–2,2.4,Manchester City,61065.0,Tottenham Hotspur Stadium,Chris Kavanagh,Match Report,NaN
410,34.0,Wed,2024-05-15,19:45,Brighton,1.3,1–2,1.5,Chelsea,31555.0,The American Express Community Stadium,Michael Salisbury,Match Report,NaN
411,34.0,Wed,2024-05-15,20:00,Manchester Utd,1.5,3–2,2.2,Newcastle Utd,73582.0,Old Trafford,Robert Jones,Match Report,NaN
412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
413,38.0,Sun,2024-05-19,16:00,Luton Town,2.0,2–4,1.1,Fulham,12027.0,Kenilworth Road Stadium,Matt Donohue,Match Report,NaN
414,38.0,Sun,2024-05-19,16:00,Sheffield Utd,1.0,0–3,3.1,Tottenham,29116.0,Bramall Lane,Andy Madley,Match Report,NaN
415,38.0,Sun,2024-05-19,16:00,Manchester City,1.9,3–1,0.4,West Ham,55097.0,Etihad Stadium,John Brooks,Match Report,NaN
416,38.0,Sun,2024-05-19,16:00,Arsenal,2.9,2–1,0.6,Everton,60312.0,Emirates Stadium,Michael Oliver,Match Report,NaN
417,38.0,Sun,2024-05-19,16:00,Brighton,1.7,0–2,1.3,Manchester Utd,31662.0,The American Express Community Stadium,Craig Pawson,Match Report,NaN
